In [4]:
import pandas as pd
import requests
import plotly.express as px


# Fetch the data.
df = pd.read_csv("https://ourworldindata.org/grapher/gdp-per-capita-worldbank.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Fetch the metadata
metadata = requests.get("https://ourworldindata.org/grapher/gdp-per-capita-worldbank.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

In [5]:
df

,Entity,Code,Year,ny_gdp_pcap_pp_kd
0,Afghanistan,AFG,2000,1617.8264
1,Afghanistan,AFG,2001,1454.1108
2,Afghanistan,AFG,2002,1774.3087
3,Afghanistan,AFG,2003,1815.9282
4,Afghanistan,AFG,2004,1776.9182
...,...,...,...,...
7058,Zimbabwe,ZWE,2019,3294.8062
7059,Zimbabwe,ZWE,2020,2987.2698
7060,Zimbabwe,ZWE,2021,3184.7854
7061,Zimbabwe,ZWE,2022,3323.1218


In [6]:
df2=df[df["Year"]==2023]
df2

,Entity,Code,Year,ny_gdp_pcap_pp_kd
23,Afghanistan,AFG,2023,1992.4244
57,Albania,ALB,2023,18244.2930
91,Algeria,DZA,2023,15159.3240
125,Andorra,AND,2023,64631.2970
159,Angola,AGO,2023,7244.8930
...,...,...,...,...
6926,Vanuatu,VUT,2023,3118.3960
6960,Vietnam,VNM,2023,13491.8800
6994,World,OWID_WRL,2023,20670.9410
7028,Zambia,ZMB,2023,3673.4841


In [7]:
datos=pd.read_csv("OECD,DF_BLI.csv")
datos

FileNotFoundError: [Errno 2] No such file or directory: 'OECD,DF_BLI.csv'

In [ ]:
valor_japon = datos[
    (datos["Indicator"] == "Self-reported health") &
    (datos["LOCATION"] == "JPN")
]["OBS_VALUE"].values[0]

valor_japon

37.0

In [ ]:

indicadores_negativos = [
    'Air pollution',
    'Dwellings without basic facilities',
    'Employees working very long hours',
    'Homicide rate',
    'Housing expenditure',
    'Labour market insecurity',
    'Long-term unemployment rate'
]

datos_filtrados = datos[["Country", "Indicator", "OBS_VALUE", "INDICATOR"]].dropna()

def obtener_dominio(codigo):
    if codigo.startswith("HO_"):
        return "Housing"
    elif codigo.startswith("JE_"):
        return "Jobs"
    elif codigo.startswith("EQ_"):
        return "Environment"
    elif codigo.startswith("HS_"):
        return "Health"
    elif codigo.startswith("ES_"):
        return "Education"
    elif codigo.startswith("SC_"):
        return "Community"
    elif codigo.startswith("PS_"):
        return "Safety"
    elif codigo.startswith("SW_"):
        return "Life Satisfaction"
    elif codigo.startswith("WL_"):
        return "Work-Life Balance"
    elif codigo.startswith("CG_"):
        return "Civic Engagement"
    elif codigo.startswith("IW_"):
        return "Income"
    else:
        return "otros"

datos_filtrados["Domain"] = datos_filtrados["INDICATOR"].apply(obtener_dominio)

promedios_finales = []

for dominio in datos_filtrados["Domain"].unique():
    subset_dominio = datos_filtrados[datos_filtrados["Domain"] == dominio]
    indicadores_norm = []

    for indicador in subset_dominio["Indicator"].unique():
        sub = subset_dominio[subset_dominio["Indicator"] == indicador].copy()
        min_val = sub["OBS_VALUE"].min()
        max_val = sub["OBS_VALUE"].max()

        if indicador in indicadores_negativos:
            sub["normalized"] = 1 - (sub["OBS_VALUE"] - min_val) / (max_val - min_val)
        else:
            sub["normalized"] = (sub["OBS_VALUE"] - min_val) / (max_val - min_val)

        indicadores_norm.append(sub[["Country", "normalized"]])

    
    combinado = pd.concat(indicadores_norm)
    promedio_dominio = combinado.groupby("Country")["normalized"].mean().reset_index()
    promedio_dominio.columns = ["Country", dominio]

    promedios_finales.append(promedio_dominio)

df_promedios_por_dominio = promedios_finales[0]
for df in promedios_finales[1:]:
    df_promedios_por_dominio = df_promedios_por_dominio.merge(df, on="Country", how="outer")


df_promedios_por_dominio



,Country,Jobs,Housing,Safety,Community,Education,Environment,Civic Engagement,Health,Work-Life Balance,Life Satisfaction,Income
0,Australia,0.858945,0.629921,0.766243,0.761538,0.768147,0.871572,0.875000,0.861567,0.408961,0.680952,0.596762
1,Austria,0.846546,0.624134,0.905345,0.730769,0.617556,0.805379,0.354787,0.713646,0.553274,0.709524,0.464477
2,Belgium,0.780398,0.750226,0.683707,0.623077,0.701194,0.555268,0.657447,0.729331,0.719235,0.614286,0.519196
3,Brazil,0.500000,0.813370,0.417698,0.423077,0.250418,0.400557,0.622340,0.567164,0.835866,0.480952,NaN
4,Canada,0.843819,0.782920,0.839247,0.738462,0.716491,0.835173,0.669681,0.863726,0.589823,0.652381,0.531429
5,Chile,0.470416,0.669774,0.560185,0.584615,0.441528,0.113127,0.046277,0.598561,0.777102,0.476190,0.065600
6,Colombia,0.732016,0.328691,0.413457,0.315385,0.184756,0.500028,0.135106,0.718523,0.303951,0.400000,NaN
7,Costa Rica,0.689132,0.626610,0.529237,0.400000,0.192476,0.664437,0.373404,0.714019,0.368794,0.528571,0.171709
8,Czechia,0.819596,0.537844,0.842087,0.869231,0.693662,0.708333,0.280851,0.592404,0.869301,0.652381,0.414408
9,Denmark,0.866818,0.623802,0.902564,0.807692,0.717738,0.848523,0.625532,0.698188,0.969605,0.766667,0.333200


In [ ]:
fig = px.scatter(df_promedios_por_dominio,x="Income",y="Life Satisfaction",size="Health",color="Country",hover_name="Country",size_max=40,title="Correlación entre Ingreso, Satisfacción con la Vida y Salud")

fig.update_layout(template="plotly_white")
fig.show()

In [ ]:
fig = px.bar(df_promedios_por_dominio,x="Country",y="Jobs",title="Comparación del Dominio 'Jobs' entre Países",color="Jobs",color_continuous_scale="Reds")
fig.show()

In [ ]:

fig = px.bar(
    df_promedios_por_dominio.sort_values("Total", ascending=False),
    x="Country",
    y="Total",
    title="Ranking Global de Calidad de Vida por País",
    color="Total",
    color_continuous_scale="Viridis"
)
fig.show()